In [70]:
import numpy as np
import math
import random as rd

In [71]:
def activation_func(x):
    return ((np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x)))

In [72]:
def perseptron_layers():
    return 2

In [73]:
def input_layer_amount():
    return 3

In [74]:
def neuron_amount():
    amount = 4 #int(input())
    return amount

In [75]:
def max_neuron_amount(neuron_amount):
    return 4

In [76]:
def weights_amount(input_layer=3,output_layer=1,neurons=4):
    half_neurons1 = int(neurons/2)
    half_neurons2 = neurons-half_neurons1
    weights_amount = input_layer * half_neurons2 + half_neurons2*half_neurons1 + half_neurons1*output_layer
    return weights_amount

In [77]:
def weights_init(weights_amount):
    weights = np.random.rand(1,weights_amount)
    return weights

In [78]:
#counts one summator
def get_summator(input_array, weights):
    result = 0
    for i in range(0,len(input_array)):
        result += (input_array[i]*weights[i])
        #print(input_array[i],"*",weights[i],"+=", result)
    #print("summator result:", result)
    return result

In [79]:
#counts summators for the first layer
def first_summators(x_array, weights, neuron_num):
    summators=list()
    layer_fin = neuron_num-int(neuron_num/2)
    for i in range(0,len(weights),len(x_array)):
        if layer_fin!=0:
            stop = i+len(x_array)
            if len(weights)>stop:
                summators.append(get_summator(x_array, weights[i:stop]))
                layer_fin-=1
    return summators

In [80]:
def second_summators(x_array, weights, neuron_num):
    summators=list()
    layer_fin = int(neuron_num/2)
    start = int(neuron_num/2)*input_layer_amount()
    for i in range(start,len(weights),len(x_array)):
        if layer_fin!=0:
            stop = i+len(x_array)
            if len(weights)>stop:
                summators.append(get_summator(x_array, weights[i:stop]))
                layer_fin-=1
    return summators

In [81]:
def third_summators(x_array, weights, neuron_num):
    summators=list()
    start = int(neuron_num/2)*input_layer_amount()+int(neuron_num/2)*len(x_array)
    summators.append(get_summator(x_array, weights[start:]))
    return summators

In [82]:
def error(network_answers, y_array):
    error = 0
    for i in range(0,len(y_array)):
        error += (network_answers[i]-y_array[i][0])**2
        #print(network_answers[i],"-",y_array[i],"+=",error)
    #print(error)
    return error

In [83]:
def neuron_error(error,weight,activated):
    counted_error = 0
    counted_error = (error*weight)*(activated*(1-activated))
    return counted_error

In [84]:
def get_second_layer_neuron_errors(network_answers, error, y, weights, activateds):
    errors_list = neuron_error(error,y,error)
    for i in range(0,int(neuron_amount()/2)):
        errors_list[i]=(float(neuron_error(errors_list[-1],weights[-2+i],activateds[-2+i])))
    return errors_list

In [85]:
def get_first_layer_neuron_errors(second_layer_errors, weights, activateds):
    errors_list = list()
    value=0
    for i in range(0,int(neuron_amount()/2)):
        error = (second_layer_errors[-2]*weights[value+(neuron_amount()-int(neuron_amount()/2)
                *input_layer_amount())]+second_layer_errors[-1]*weights[value+(neuron_amount()
                -int(neuron_amount()/2)+2)])*(activateds[-2+i]*(1-activateds[-2+i]))
        value+=2
        errors_list.append(float(error))
    return errors_list

In [86]:
def correction(x, error, weight):
    corrected = 0.0001 * error * x
    corrected += weight
    return corrected

In [87]:
def first_layer_weights_correction(x_array, first_layer_err, weights):
    fin=neuron_amount()-int(neuron_amount()/2)
    j=0
    k=0
    for i in range(0,(len(x_array)*fin)):#по кол-ву weights в 1-м слое
        if j>=len(x_array):
            j=0
            k+=1
        weights[i]=correction(x_array[j], first_layer_err[k], weights[i])
        j+=1
    return weights

In [88]:
def second_layer_weights_correction(activateds, second_layer_err, weights):
    start=int(neuron_amount()/2)*input_layer_amount()
    layer_fin = int(neuron_amount()/2)
    j=0
    k=0
    for i in range(start,len(weights)):
        if len(second_layer_err)-1>k:
            if j>=len(activateds):
                j=0
                k+=1
            weights[i]=correction(activateds[j],second_layer_err[k],weights[i])
            j+=1
    return weights

In [89]:
def third_layer_weights_correction(activateds, third_layer_err, weights):
    start=int(neuron_amount()/2)*input_layer_amount()+int(neuron_amount()/2)*(neuron_amount()-int(neuron_amount()/2))
    j=0
    for i in range(start,len(weights)):
        if len(activateds)>j:
            weights[i]=correction(activateds[j],third_layer_err,weights[i])
            j+=1
    return weights

In [90]:
#initialisation
x_array = np.array([[2,1,3],[1,2,1],[3,2,1],[1,3,2]])
y_array = np.array([[1],[0.5],[1],[0.5]])
weights = weights_init(weights_amount())
print(weights)

[[0.21058924 0.17093012 0.93878612 0.55887554 0.21217115 0.76886744
  0.42951957 0.44098718 0.21272511 0.5304365  0.09711035 0.23175186]]


In [91]:
min_error=999.0
min_error_weights=None
network_answers=list()
res_first_layer=list()
res_second_layer=list()
res_third_layer=list()
for test in range(0,6000):
    pos=0
    for j in range(0,len(x_array)):
        #network_answers=list()
        res_first_layer=list()
        res_second_layer=list()
        res_third_layer=list()
        print("*"*10,"Learning data:",x_array[j],"*"*10)
        #print(weights)
        #get first layer
        res_first_layer=first_summators(x_array[j], weights[0], neuron_amount())
        if res_first_layer==float('nan'):
            break
        #print("first layer:",res_first_layer)
        #activation
        for i in range(0,len(res_first_layer)):
            res_first_layer[i] = activation_func(res_first_layer[i])
        #print("activation:",res_first_layer)  

        #get second layer
        res_second_layer=second_summators(res_first_layer, weights[0], neuron_amount())
        #print("second layer:",res_second_layer)
        for i in range(0,len(res_second_layer)):
            res_second_layer[i] = activation_func(res_second_layer[i])
        #print("activation:",res_second_layer) 

        #get third layer
        res_third_layer=third_summators(res_second_layer, weights[0], neuron_amount())
        #print("third layer:",res_third_layer)
        for i in range(0,len(res_third_layer)):
            res_third_layer[i] = activation_func(res_third_layer[i])
        print("activation:",res_third_layer)
        network_answers.append(res_third_layer[0])
    print("*"*30)    
    print("Network answers:",network_answers)
    
    network_error = error(network_answers, y_array)        
    print("Network error:",network_error)
    if min_error>network_error:
        min_error=network_error
        min_error_weights=weights
    second_layer_errors=get_second_layer_neuron_errors(network_answers, network_error, y_array, weights[0], res_second_layer)
    #print("Second layer errors:")
    for i in second_layer_errors:
        #print("{:1.16f}".format(i))
        #print(i)
        pass
    first_layer_errors=get_first_layer_neuron_errors(second_layer_errors, weights[0], res_first_layer)
    #print("First layer errors:")
    for i in first_layer_errors:
        #print("{:1.16f}".format(i))
        pass
    first_layer_weights_correction(x_array[pos],first_layer_errors,weights[0])
    if pos>=len(x_array)-1:
        pos=0
    else:
        pos+=1
    second_layer_weights_correction(res_first_layer, second_layer_errors, weights[0])
    third_layer_weights_correction(res_second_layer, network_error, weights[0])
    #print(weights)
    network_answers=list()
    #else: 
        #print("Min-error found:",min_error)
        #break
print("Min_error",min_error)
print("Min weights",min_error_weights)
weights=min_error_weights

#Min_error 0.2481654673027066 при обучении 0.0005 за 3000 эпохи
#Min_error 0.2358068490491364 при обучении 0.0001 за 3000 эпохи
#Min_error 0.2540872679395598 при обучении 0.000001
#Min_error 0.24870568510740332 при обучении 0.00001

********** Learning data: [2 1 3] **********
activation: [0.21092463929624708]
********** Learning data: [1 2 1] **********
activation: [0.20073445807942117]
********** Learning data: [3 2 1] **********
activation: [0.20846723041458834]
********** Learning data: [1 3 2] **********
activation: [0.20984947256222097]
******************************
Network answers: [0.21092463929624708, 0.20073445807942117, 0.20846723041458834, 0.20984947256222097]
Network error: 1.4229112433507491
********** Learning data: [2 1 3] **********
activation: [0.21093626978729155]
********** Learning data: [1 2 1] **********
activation: [0.20074549749052062]
********** Learning data: [3 2 1] **********
activation: [0.20847869864422677]
********** Learning data: [1 3 2] **********
activation: [0.2098610405338895]
******************************
Network answers: [0.21093626978729155, 0.20074549749052062, 0.20847869864422677, 0.2098610405338895]
Network error: 1.422861413909404
********** Learning data: [2 1 3] ***

activation: [0.2105468849437348]
******************************
Network answers: [0.211625800973338, 0.20140014098071013, 0.20915867347734277, 0.2105468849437348]
Network error: 1.4199088630491623
********** Learning data: [2 1 3] **********
activation: [0.21163837575584823]
********** Learning data: [1 2 1] **********
activation: [0.20141208221805726]
********** Learning data: [3 2 1] **********
activation: [0.20917107508399027]
********** Learning data: [1 3 2] **********
activation: [0.21055939283403227]
******************************
Network answers: [0.21163837575584823, 0.20141208221805726, 0.20917107508399027, 0.21055939283403227]
Network error: 1.4198550487866493
********** Learning data: [2 1 3] **********
activation: [0.21165096740314457]
********** Learning data: [1 2 1] **********
activation: [0.20142403956221142]
********** Learning data: [3 2 1] **********
activation: [0.20918349336064362]
********** Learning data: [1 3 2] **********
activation: [0.21057191751106308]
****

********** Learning data: [2 1 3] **********
activation: [0.21237007452123938]
********** Learning data: [1 2 1] **********
activation: [0.202107066475753]
********** Learning data: [3 2 1] **********
activation: [0.20989275842488517]
********** Learning data: [1 3 2] **********
activation: [0.2112872183613657]
******************************
Network answers: [0.21237007452123938, 0.202107066475753, 0.20989275842488517, 0.2112872183613657]
Network error: 1.4167256228243141
********** Learning data: [2 1 3] **********
activation: [0.212383643157717]
********** Learning data: [1 2 1] **********
activation: [0.2021199569259289]
********** Learning data: [3 2 1] **********
activation: [0.2099061424159596]
********** Learning data: [1 3 2] **********
activation: [0.2113007155087462]
******************************
Network answers: [0.212383643157717, 0.2021199569259289, 0.2099061424159596, 0.2113007155087462]
Network error: 1.4166676262851128
********** Learning data: [2 1 3] **********
activ

Network error: 1.4134210436862977
********** Learning data: [2 1 3] **********
activation: [0.21315828155330416]
********** Learning data: [1 2 1] **********
activation: [0.20285602243676848]
********** Learning data: [3 2 1] **********
activation: [0.21067029843159912]
********** Learning data: [1 3 2] **********
activation: [0.21207129095720112]
******************************
Network answers: [0.21315828155330416, 0.20285602243676848, 0.21067029843159912, 0.21207129095720112]
Network error: 1.413358752559361
********** Learning data: [2 1 3] **********
activation: [0.2131728929702891]
********** Learning data: [1 2 1] **********
activation: [0.20286990888988485]
********** Learning data: [3 2 1] **********
activation: [0.21068471318883805]
********** Learning data: [1 3 2] **********
activation: [0.21208582606830853]
******************************
Network answers: [0.2131728929702891, 0.20286990888988485, 0.21068471318883805, 0.21208582606830853]
Network error: 1.4132963809444046
***

********** Learning data: [2 1 3] **********
activation: [0.2140382182365223]
********** Learning data: [1 2 1] **********
activation: [0.20369245621505838]
********** Learning data: [3 2 1] **********
activation: [0.2115384552500969]
********** Learning data: [1 3 2] **********
activation: [0.21294665207990562]
******************************
Network answers: [0.2140382182365223, 0.20369245621505838, 0.2115384552500969, 0.21294665207990562]
Network error: 1.4096053149982237
********** Learning data: [2 1 3] **********
activation: [0.21405398177190518]
********** Learning data: [1 2 1] **********
activation: [0.203707443180647]
********** Learning data: [3 2 1] **********
activation: [0.21155400887566947]
********** Learning data: [1 3 2] **********
activation: [0.2129623339956587]
******************************
Network answers: [0.21405398177190518, 0.203707443180647, 0.21155400887566947, 0.2129623339956587]
Network error: 1.4095381254203938
********** Learning data: [2 1 3] **********

...

activation: [0.5039366106523787]
********** Learning data: [3 2 1] **********
activation: [0.5199992604328066]
********** Learning data: [1 3 2] **********
activation: [0.5231287206591275]
******************************
Network answers: [0.5253393100514787, 0.5039366106523787, 0.5199992604328066, 0.5231287206591275]
Network error: 0.4562539151902152
********** Learning data: [2 1 3] **********
activation: [0.5253796476617726]
********** Learning data: [1 2 1] **********
activation: [0.5039761345723912]
********** Learning data: [3 2 1] **********
activation: [0.5200393966047152]
********** Learning data: [1 3 2] **********
activation: [0.5231689815575642]
******************************
Network answers: [0.5253796476617726, 0.5039761345723912, 0.5200393966047152, 0.5231689815575642]
Network error: 0.4561792710177815
********** Learning data: [2 1 3] **********
activation: [0.5254199761545955]
********** Learning data: [1 2 1] **********
activation: [0.5040156497700219]
********** Learni

********** Learning data: [2 1 3] **********
activation: [0.5277833415290247]
********** Learning data: [1 2 1] **********
activation: [0.5063317233425841]
********** Learning data: [3 2 1] **********
activation: [0.5224311878117596]
********** Learning data: [1 3 2] **********
activation: [0.5255681456052362]
******************************
Network answers: [0.5277833415290247, 0.5063317233425841, 0.5224311878117596, 0.5255681456052362]
Network error: 0.4517543637025581
********** Learning data: [2 1 3] **********
activation: [0.5278231291762044]
********** Learning data: [1 2 1] **********
activation: [0.5063707210605055]
********** Learning data: [3 2 1] **********
activation: [0.522470780111781]
********** Learning data: [1 3 2] **********
activation: [0.5256078589581753]
******************************
Network answers: [0.5278231291762044, 0.5063707210605055, 0.522470780111781, 0.5256078589581753]
Network error: 0.45168150171525484
********** Learning data: [2 1 3] **********
activa

********** Learning data: [1 3 2] **********
activation: [0.5277392910013374]
******************************
Network answers: [0.5299585156248102, 0.5084640165998633, 0.5245957626796675, 0.5277392910013374]
Network error: 0.44778929373801846
********** Learning data: [2 1 3] **********
activation: [0.5299978180602218]
********** Learning data: [1 2 1] **********
activation: [0.5085025499501389]
********** Learning data: [3 2 1] **********
activation: [0.5246348751094839]
********** Learning data: [1 3 2] **********
activation: [0.5277785212619727]
******************************
Network answers: [0.5299978180602218, 0.5085025499501389, 0.5246348751094839, 0.5277785212619727]
Network error: 0.44771799258948475
********** Learning data: [2 1 3] **********
activation: [0.5300371117657509]
********** Learning data: [1 2 1] **********
activation: [0.5085410749445332]
********** Learning data: [3 2 1] **********
activation: [0.5246739789052443]
********** Learning data: [1 3 2] **********
act

********** Learning data: [1 3 2] **********
activation: [0.5298065931601722]
******************************
Network answers: [0.5320295910854653, 0.5104948418216985, 0.5266568988278932, 0.5298065931601722]
Network error: 0.44404856974754264
********** Learning data: [2 1 3] **********
activation: [0.5320684353093701]
********** Learning data: [1 2 1] **********
activation: [0.5105329365419359]
********** Learning data: [3 2 1] **********
activation: [0.5266955580625432]
********** Learning data: [1 3 2] **********
activation: [0.5298453671991928]
******************************
Network answers: [0.5320684353093701, 0.5105329365419359, 0.5266955580625432, 0.5298453671991928]
Network error: 0.4439787326869997
********** Learning data: [2 1 3] **********
activation: [0.5321072709747994]
********** Learning data: [1 2 1] **********
activation: [0.5105710230684904]
********** Learning data: [3 2 1] **********
activation: [0.5267342088321545]
********** Learning data: [1 3 2] **********
acti

activation: [0.528656023792987]
********** Learning data: [1 3 2] **********
activation: [0.5318116137813294]
******************************
Network answers: [0.5340382125702002, 0.5124649690718575, 0.528656023792987, 0.5318116137813294]
Network error: 0.44045288547674594
********** Learning data: [2 1 3] **********
activation: [0.5340766159688304]
********** Learning data: [1 2 1] **********
activation: [0.5125026417110352]
********** Learning data: [3 2 1] **********
activation: [0.52869424700269]
********** Learning data: [1 3 2] **********
activation: [0.531849948898792]
******************************
Network answers: [0.5340766159688304, 0.5125026417110352, 0.52869424700269, 0.531849948898792]
Network error: 0.4403844478900284
********** Learning data: [2 1 3] **********
activation: [0.5341150109737202]
********** Learning data: [1 2 1] **********
activation: [0.5125403063124835]
********** Learning data: [3 2 1] **********
activation: [0.528732461909811]
********** Learning data:

In [95]:
#test
x_test = np.array([[1,2,1]])
y_test = np.array([[0.5]])


network_answers=list()
res_first_layer=list()
res_second_layer=list()
res_third_layer=list()
print("*"*10,"Learning data:",x_test[0],"*"*10)
print(weights)
#get first layer
res_first_layer=first_summators(x_test[0], weights[0], neuron_amount())
print("first layer:",res_first_layer)
#activation
for i in range(0,len(res_first_layer)):
    res_first_layer[i] = activation_func(res_first_layer[i])
print("activation:",res_first_layer)  

#get second layer
res_second_layer=second_summators(res_first_layer, weights[0], neuron_amount())
print("second layer:",res_second_layer)
for i in range(0,len(res_second_layer)):
    res_second_layer[i] = activation_func(res_second_layer[i])
print("activation:",res_second_layer) 

#get third layer
res_third_layer=third_summators(res_second_layer, weights[0], neuron_amount())
print("third layer:",res_third_layer)
for i in range(0,len(res_third_layer)):
    res_third_layer[i] = activation_func(res_third_layer[i])
print("activation:",res_third_layer)
network_answers.append(res_third_layer)

print("*"*30)    
print("Network answers:",network_answers[0][0])

********** Learning data: [1 2 1] **********
[[0.21045558 0.17086329 0.93858564 0.55840093 0.21193385 0.76815553
  0.42961462 0.44108245 0.21172777 0.52943659 0.40480283 0.50057956]]
first layer: [1.4907678055069895, 1.7504241568558174]
activation: [0.9034659242109081, 0.941423793407237]
second layer: [0.8033876855684435, 0.6897130303817928]
activation: [0.6659264153159324, 0.5977976145504067]
third layer: [0.5688141648943578]
activation: [0.51448786285608]
******************************
Network answers: 0.51448786285608


1